Introduction
============

Recently, I started using the [pandas](http://pandas.pydata.org/) python library to improve the quality 
(and quantity) of statistics in my applications. One method that has helped me much is the `pivot_table`
command. This is a really (at least for me) complicated command that has
[very poor documentation](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html).
Beyond this, this command is explained a little more in an article about 
[data reshaping](http://pandas.pydata.org/pandas-docs/stable/reshaping.html), however, even this leaves
much to be desired (when I first tried reading it I was overwhelmed by the amount of information there).

A great introduction to pandas is the [three part series](http://www.gregreda.com/2013/10/26/intro-to-pandas-data-structures/) by Greg Reda - it touches `pivot_table` however I was only able to understand it properly *after* I loose a lot of time playing myself with `pivot_table`.

I have to mention that I am no expert in statistics or
numeric analysis so this post won't have any advanced 
information and may even point out some obvious things. However
keep in mind things that may seem obvious to some
experts are really difficult to grasp for a non-expert.

Before continuing, please notice that this article has been written as a [jupyter notebook](http://jupyter.org/) and was integrated with pelican using the [pelican-ipynb plugin](https://github.com/danielfrg/pelican-ipynb). I had to do some modifications to my theme to better integrate the notebook styling, however some stuff may not look as nice as the other articles.

The DataFrame
=============

The most important (new) data structure that pandas uses is the [DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html). This can be thought as a two dimensional array, something like an Excel sheet. In the pandas nomenclature, the rows of that two-dimensional array are called *indexes* (while the columns keep their name). Also, if we horizontally/vertically pick the values of a single row(index)/column we'll be left with a different data structure called [Series](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.html) - so an array is a series. There's also a [Panel](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Panel.html) data structure which is 3-dimensional, more like a complete Excel workbook (the third dimension being the sheets of the workbook) but I won't cover that here.

More info on the above can be found on the corresponding [article about data structures](http://pandas.pydata.org/pandas-docs/stable/dsintro.html).

There are various ways to read the data for a Series or DataFrame: Initializing through arrays or dicts, reading from csv, xls, database, combinining series to create an array and various others. I won't go into any details about this but will include some examples on how to create Series and DataFrames.

Before continuing with more info on the `pivot_table` operation, I will list a number of examples as a quick reference on how to work with Series and DataFrames that will be useful in the rest of the article (and should be useful in general). Notice that I'm using pandas 0.18.1.

Using Series
------------

In [1]:
import pandas as pd

def t(o):
    # Return the class name of the object
    return o.__class__.__name__

# Use an array to create a Series
series1 = pd.Series([10,20,30])
print "series1 (", t(series1), ')\n', series1
# Notice that the index names were automatically generated as 0,1,2
# Use a dict to create a Series
# notice that the keys of the dict will be the index names
series2 = pd.Series({'row1':11,'row2':22,'row3':33})
print "series2 (", t(series2), ')\n', series2

series1 ( Series )
0    10
1    20
2    30
dtype: int64
series2 ( Series )
row1    11
row2    22
row3    33
dtype: int64


In [71]:
# Get values from series using named indexes
series2['row1']
# Can also use slicing and interesting operations 
# like array in array [[]] to select specific indexes
print series1[1:]
print series1[[0,2]]
print series2['row2':]
print series2[['row1', 'row3']]

1    20
2    30
dtype: int64
0    10
2    30
dtype: int64
row2    22
row3    33
dtype: int64
row1    11
row3    33
dtype: int64


In [81]:
# Filtering series
# You can use comparison operators with a Series to 
# get an array of booleans with the result of each element
print "Boolean result\n", series2>15
# This can then be used to filter the Series:
print "Filtered result\n", series2[series2>15]

Boolean result
row1    False
row2     True
row3     True
dtype: bool
Filtered result
row2    22
row3    33
dtype: int64


In [90]:
# Filtering can be generalized - just pass an array of booleans
print "Boolean filtering\n", series2[[True, False,False]]
# So we can create a function that returns Boolean, apply it to 
# all elements of series with map and use the result for indexing!
def is_22(x):
    return x==22
print "Map filtering\n", series2[series2.map(is_22)]

Boolean filtering
row1    11
dtype: int64
Map filtering
row2    22
dtype: int64


Using DataFrames
----------------

Let's start by a quick introduction to see some basic operations on DataFrames:

In [116]:
# Create a DataFrame using a two-dimensional array
# Notice that the indexes and column names were automatically generated
df1 = pd.DataFrame([[10,20,30], [40,50,60]])
print "Dataframe from array: df1(", t(df1), ')'
print df1

# Use a dict to give names to 
df2 = pd.DataFrame([{'col1':10,'col2':20,'col3':30}, {'col1':40,'col2':50,'col3':60}])
print "Dataframe from dict: df2(", t(df2), ')'
print df2

# Give names to indexes
df3 = pd.DataFrame([
    {'col1':10,'col2':20,'col3':30}, 
    {'col1':40,'col2':50,'col3':60}
], index=['idx1', 'idx2'])
print "Dataframe from dict, named indexes: df3(", t(df3), ')'
print df3

# What happens when columns are missing
df4 = pd.DataFrame([{'col1':10,'col2':20,'col3':30}, {'col2':40,'col3':50,'col4':60}])
print "Dataframe from dict, missing columns: df4(", t(df4), ')'
print df4

# Create a DataFrame by combining series
df5 = pd.DataFrame([pd.Series([1,2]), pd.Series([3,4])], index=['a', 'b'], )
print "Dataframe from series: df5(", t(df5), ')'
print df5

# Output a dataframe as html
print df5.to_html()

# Notice that there are many more interesting DataFrame output methods, like
# to_csv, to_dict, to_excel, to_json, to_latex, to_msgpack, to_string,

Dataframe from array: df1( DataFrame )
    0   1   2
0  10  20  30
1  40  50  60
Dataframe from dict: df2( DataFrame )
   col1  col2  col3
0    10    20    30
1    40    50    60
Dataframe from dict, named indexes: df3( DataFrame )
      col1  col2  col3
idx1    10    20    30
idx2    40    50    60
Dataframe from dict, missing columns: df4( DataFrame )
   col1  col2  col3  col4
0  10.0    20    30   NaN
1   NaN    40    50  60.0
Dataframe from series: df5( DataFrame )
   0  1
a  1  2
b  3  4
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>0</th>
      <th>1</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>a</th>
      <td>1</td>
      <td>2</td>
    </tr>
    <tr>
      <th>b</th>
      <td>3</td>
      <td>4</td>
    </tr>
  </tbody>
</table>


Selecting values from the Dataframe is also easy if you know how to do it:

In [115]:
print "df3(", t(df3), ")\n", df3

# Pick values from a dataframe using array indexing
print "Get value\n", df3['col2']['idx2']

# We can get a column as a series
print "Get column as series\n", df3['col3']

# Or multiple columns (as a DatFrame)
print "Get column as series\n", df3[['col3', 'col2']]

# We can also get the column by its idx
print "Get column by index\n", df3[df3.columns[2]]

# Pick an index (select horizontal line) as a series
print "Get index as a series\n", df3.loc['idx1']

# Also can pick by index number
print "Get index as a series by index\n", df3.iloc[0]

df3( DataFrame )
      col1  col2  col3
idx1    10    20    30
idx2    40    50    60
Get value
50
Get column as series
idx1    30
idx2    60
Name: col3, dtype: int64
Get column as series
      col3  col2
idx1    30    20
idx2    60    50
Get column by index
idx1    30
idx2    60
Name: col3, dtype: int64
Get index as a series
col1    10
col2    20
col3    30
Name: idx1, dtype: int64
Get index as a series by index
col1    10
col2    20
col3    30
Name: idx1, dtype: int64


Of course, boolean indexing can also be used just like in Series:

In [127]:
print "Boolean indexing\n",df3[df3>=30]
def is_20_or_40(x):
    return x==20 or x==40
# We need to use applymap (instead of map we used in Series)
print "Boolean indexing\n",df3[df3.applymap(is_20_or_40)]

Boolean indexing
      col1  col2  col3
idx1   NaN   NaN    30
idx2  40.0  50.0    60
Boolean indexing
      col1  col2  col3
idx1   NaN  20.0   NaN
idx2  40.0   NaN   NaN


Modifying DataFrames
--------------------

It's easy to modify the DataFrame by changing its values, adding more indexes / columns, dropping rows and columns, renaming columns and indexes. Notice that some operations are performed in place (so they modify the original DataFrame), while others return a copy of the original array.

In [112]:
# Let's copy because some of the following operators change the dataframes
df = df3.copy()
print df

print "Change values of a column"
df['col1'] = [11,41]
print df

print "Change values of an index"
df.loc['idx1'] = [11,21, 31]
print df

print "Add another column to an existing dataframe (changes DataFrame)"
df['col4'] = [1,2]
print df

print "Add another row (index) to an existing dataframe (changes DataFrame)"
df.loc['idx3']=[100,200,300,400]
print df

print "Drop a row (returns new object)"
print df.drop('idx1')

print "Drop a column (returns new object)"
print df.drop('col1', axis=1)

print "Rename index (returns new object)"
print df.rename(index={'idx1': 'new-idx-1'})

print "Rename column (returns new object)"
print df.rename(columns={'col1': 'new-col-1'})

print "Transpose array- change columns to rows and vice versa"
print df.T

print "Double transpose - returns the initial DataFrame"
print df.T.T

      col1  col2  col3
idx1    10    20    30
idx2    40    50    60
Change values of a column
      col1  col2  col3
idx1    11    20    30
idx2    41    50    60
Change values of an index
      col1  col2  col3
idx1    11    21    31
idx2    41    50    60
Add another column to an existing dataframe (changes DataFrame)
      col1  col2  col3  col4
idx1    11    21    31     1
idx2    41    50    60     2
Add another row (index) to an existing dataframe (changes DataFrame)
      col1  col2  col3  col4
idx1    11    21    31     1
idx2    41    50    60     2
idx3   100   200   300   400
Drop a row (returns new object)
      col1  col2  col3  col4
idx2    41    50    60     2
idx3   100   200   300   400
Drop a column (returns new object)
      col2  col3  col4
idx1    21    31     1
idx2    50    60     2
idx3   200   300   400
Rename index (returns new object)
           col1  col2  col3  col4
new-idx-1    11    21    31     1
idx2         41    50    60     2
idx3        100   200  

More advanced operations
------------------------

Beyond the previous, more or less basic operations, pandas allows you to do some more advanced operations like SQL-like joins of more than one dataset or, applying a function to each of the rows / columns or even individual cells of the DataFrame:

In [113]:
authors_df=pd.DataFrame([{'id': 1, 'name':'Stephen King'}, {'id': 2, 'name':'Michael Crichton'}],  )

books_df=pd.DataFrame([
        {'id': 1, 'author_id':1, 'name':'It'}, 
        {'id': 2, 'author_id':1, 'name':'The Stand'}, 
        {'id': 3, 'author_id':2, 'name':'Airframe'},
        {'id': 4, 'author_id':2, 'name':'Jurassic Park'}
    ])

print authors_df
print books_df
print books_df.merge(authors_df, left_on='author_id', right_on='id')

   id              name
0   1      Stephen King
1   2  Michael Crichton
   author_id  id           name
0          1   1             It
1          1   2      The Stand
2          2   3       Airframe
3          2   4  Jurassic Park
   author_id  id_x         name_x  id_y            name_y
0          1     1             It     1      Stephen King
1          1     2      The Stand     1      Stephen King
2          2     3       Airframe     2  Michael Crichton
3          2     4  Jurassic Park     2  Michael Crichton


As can be seen above, the `merge` method of DataFrame can be used to do an sql-like join with another DataFrame, using specific columns as join-keys for each of the two dataframes (`left_on` and `right_on`). There are a lot of options for doing various join types (left, right, inner, outer etc) and concatenating DataFrames with other ways - most are discussed in the [corresponding post](http://pandas.pydata.org/pandas-docs/stable/merging.html).

Let's see another method of doing the above join that is more controlled, using the `apply` method of DataFrame that *applies* a function to each row/column of the DataFrame and returns the result as a series:

In [94]:
# Let's do the join using a different method
def f(r):
    author_df_partial = authors_df[authors_df.id==r['author_id']]
    
    return author_df_partial.iloc[0]['name']
    
books_df['author name'] = books_df.apply(f, axis=1)
print books_df

   author_id  id           name       author name
0          1   1             It      Stephen King
1          1   2      The Stand      Stephen King
2          2   3       Airframe  Michael Crichton
3          2   4  Jurassic Park  Michael Crichton


How does this work? We pass the `axis=1` parameter to `apply` so that the callback function will be called for each row of the DataFrame (by default `axis=0` which means it will be called for each column). So, `f` will be called getting each row as an input. From this `book_df` row, we get the `author_id` it contains and filter `authors_df` by it. Notice that `author_df_partial` is actually a DataFrame containing only one row, so we need to filter it by getting its only line, using `iloc[0]` which will return a Series and finally, we return the author name using the corresponding index.

In [156]:
# Let's see another example of apply
values = pd.DataFrame([
    {'temperature': 31, 'moisture': 68},
    {'temperature': 33, 'moisture': 72},
    {'temperature': 31.5, 'moisture': 58},
    {'temperature': 28.5, 'moisture': 42},
])

import numpy as np
# We can easily create statistics for our data using apply -- that's why
# axis=0 is the default parameter to apply (to operate vertically to each column)
values.loc['avg']=values.apply(np.average )
values.loc['len']=values.apply(len )
values.loc['sum']=values.apply(sum)
print values

     moisture  temperature
0        68.0         31.0
1        72.0         33.0
2        58.0         31.5
3        42.0         28.5
avg      60.0         31.0
len       5.0          5.0
sum     305.0        160.0


Comprehending `pivot_table`
=========================

The `pivot_table` methods is applied to a DataFrame and its purpose is to "reshape" the DataFrame. More on reshaping can be found [here](http://pandas.pydata.org/pandas-docs/stable/reshaping.html). Reshaping means that we want to change the indexes/columns of the DataFrame  while aggregating the values.

Let's start by creating a nice set of data we'll use for the `pivot_table` operations:

In [158]:
books_df=pd.DataFrame([
    {'author':'Stephen King', 'name':'It', 'pages': 1138, 'year': 1986, 'genre': 'Horror',},  
    {'author':'Stephen King', 'name':'The Stand', 'pages': 823, 'year': 1978, 'genre': 'Horror',}, 
    {'author':'Stephen King', 'name': 'Salem\'s Lot', 'pages': 439, 'year': 1975, 'genre': 'Horror',},
    {'author':'Stephen King', 'name': 'Misery', 'pages': 320, 'year': 1987, 'genre': 'Thriller',},
    {'author':'Stephen King', 'name': 'Pet Sematary', 'pages': 374, 'year': 1983, 'genre': 'Horror',},
    {'author':'Stephen King', 'name': 'Bag of bones', 'pages': 529, 'year': 1998, 'genre': 'Horror',},
    {'author':'Stephen King', 'name': 'Different Seasons', 'pages': 527, 'year': 1982, 'genre': 'Thriller',},
    {'author':'Stephen King', 'name': 'The Dark Tower: The Gunslinger', 'pages': 224, 'year': 1982, 'genre': 'Fantasy',},
    {'author':'Stephen King', 'name': 'The Dark Tower II: The Drawing of the Three', 'pages': 400, 'year': 1987, 'genre': 'Fantasy',},
    {'author':'Stephen King', 'name': 'The Dark Tower III: The Waste Lands', 'pages': 512, 'year': 1991, 'genre': 'Fantasy',},
    {'author':'Stephen King', 'name': 'The Dark Tower IV: Wizard and Glass', 'pages': 787, 'year': 1998, 'genre': 'Fantasy',},
    {'author':'Michael Crichton', 'name':'Airframe', 'pages': 352, 'year': 1996, 'genre': 'Crime',},
    {'author':'Michael Crichton', 'name':'Jurassic Park', 'pages': 448, 'year':1990, 'genre': 'Fantasy',},
    {'author':'Michael Crichton', 'name':'Congo', 'pages': 348, 'year':1980, 'genre': 'Fantasy',},
    {'author':'Michael Crichton', 'name':'Sphere', 'pages': 385, 'year':1987, 'genre': 'Fantasy',},
    {'author':'Michael Crichton', 'name':'Rising Sun', 'pages': 385, 'year':1992, 'genre': 'Crime',},
    {'author':'Michael Crichton', 'name':'Disclosure ', 'pages': 597, 'year':1994, 'genre': 'Crime',},
    {'author':'Michael Crichton', 'name':'The Lost World ', 'pages': 430, 'year':1995, 'genre': 'Fantasy',},
    {'author':'John Grisham', 'name':'A Time to Kill', 'pages': 515, 'year':1989, 'genre': 'Crime',},
    {'author':'John Grisham', 'name':'The Firm', 'pages': 432, 'year':1991, 'genre': 'Crime',},
    {'author':'John Grisham', 'name':'The Pelican Brief', 'pages': 387, 'year':1992, 'genre': 'Crime',},
    {'author':'John Grisham', 'name':'The Chamber', 'pages': 496, 'year':1994, 'genre': 'Crime',},
    {'author':'John Grisham', 'name':'The Rainmaker', 'pages': 434, 'year':1995, 'genre': 'Crime',},
    {'author':'John Grisham', 'name':'The Runaway Jury', 'pages': 414, 'year':1996, 'genre': 'Crime',},
    {'author':'John Grisham', 'name':'The Street Lawyer', 'pages': 347, 'year':1998, 'genre': 'Crime',},
    {'author':'George Pelecanos', 'name':'Nick\'s Trip ', 'pages': 276, 'year':1993, 'genre': 'Crime',},
    {'author':'George Pelecanos', 'name':'A Firing Offense', 'pages': 216, 'year':1992, 'genre': 'Crime',},
    {'author':'George Pelecanos', 'name':'The Big Blowdown', 'pages': 313, 'year':1996, 'genre': 'Crime',},
    {'author':'George R.R Martin', 'name':'A Clash of Kings', 'pages': 768, 'year':1998, 'genre': 'Fantasy',},
    {'author':'George R.R Martin', 'name':'A Game of Thrones', 'pages': 694, 'year':1996, 'genre': 'Fantasy',},
])

# Add a decade column to the books DataFrame
def add_decade(y):
        return str(y['year'])[2] + '0\'s'
    
books_df['decade'] = books_df.apply(add_decade, axis=1)

# Add a size column to the books DataFrame
def add_size(y):
        if y['pages'] > 600:
            return 'big'
        elif y['pages'] < 300:
            return 'small'
        return 'medium'
    
books_df['size'] = books_df.apply(add_size, axis=1)
# Let's display it sorted here
books_df.sort_values(['decade', 'genre', 'year'])

,author,genre,name,pages,year,decade,size
2,Stephen King,Horror,Salem's Lot,439,1975,70's,medium
1,Stephen King,Horror,The Stand,823,1978,70's,big
18,John Grisham,Crime,A Time to Kill,515,1989,80's,medium
13,Michael Crichton,Fantasy,Congo,348,1980,80's,medium
7,Stephen King,Fantasy,The Dark Tower: The Gunslinger,224,1982,80's,small
8,Stephen King,Fantasy,The Dark Tower II: The Drawing of the Three,400,1987,80's,medium
14,Michael Crichton,Fantasy,Sphere,385,1987,80's,medium
4,Stephen King,Horror,Pet Sematary,374,1983,80's,medium
0,Stephen King,Horror,It,1138,1986,80's,big
6,Stephen King,Thriller,Different Seasons,527,1982,80's,medium


The recommended type of input (at least by me) to the `pivot_table` is a simple DataFrame like the one I have already created:  Your index will be the `id` of your database (or you could even have an auto-generated index like in the example) and the columns will be the values you want to create representations on. This is very easy to create either by reading a file (xls/csv) or by a simple SQL query (substituting all foreign keys with a representative value). In the above example, we actually have the following columns: *author, genre, name, pages, year, decade, size* - this is a pool of data that will be very useful to remember for later and it is important to also keep it in your mind for your data.

The [pivot_table method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html) user four basic parameters:

* `index`: An array of the columns or index names that will be used as indexes to the resulting DataFrame
* `columns`: An array of the columns or index names that will be used as a columns to the resulting DataFrame
* `values`: An array of the values we want to aggregate
* `aggfunc`: Which is the function (or functions) that will be used for aggregating the values

So, how it actually works? You select a number of the headers from your pool of data and assign them to either `index` or `columns`, depending if you want to put them horizontally or vertically -- notice that both `index` and `columns` take either a string (to denote a single column) or an array to denote multiple columns. Multiple columns means that you'll have [hierachical indexes / columns](http://pandas.pydata.org/pandas-docs/stable/advanced.html) in your pivot, ie the rows/columns would be grouped by a hierarchy, depending on where they actually belong. Please notice that the intersection of index and columns sets have to be an empty set (i.e you can't use the same thing in both). Let's make it crystal with an example:

If we used `'decade'` as an index, then the pivot_table index would be like 
* `70s` value1 value2 ...
* `80s` value1 value2 ...
* `90s` value1 value2 ...

while, if we used `['decade', 'year']` we'd hove something like

* `70s`
    * `1975` value1 value2 ...
    * `1978` value1 value2 ...
* `80s`
    * `1980` value1 value2 ...
    * `1982` value1 value2 ...
    * ...
* `90s`
    * `1990` value1 value2 ...
    * ... 
    
So, each year would automatically be grouped to its corresponing decade. The same would be true if we used `['decade', 'year']` in columns. Notice that pandas doesn't know if the values have any parent / child relationship but just goes from left to right. For example, if we had used `['year', 'decade']`, we'd get something like:

* `1975 70s'` value1 value2 ... 
* `1978 70s'` value1 value2 ...
* `1980 80s'` value1 value2 ...	
* `1982 80s'` value1 value2 ...

Also, pandas doesn't care if the values of the hierarchical index are actually related. We could for example had selected a multi index of `['decade', 'size', 'genre']` that would display the list of decades at the left (or at the top if we used it as a column), for each decade will display the sizes of the book of that decade at the center and finally, at the right (or bottom correspondingly) will display the available genres for each size. 

I hope the above clears out how index and columns are used to create the headers for rows and index of `pivot_table`. I will show some examples of various index and columns combinations but first, I'd like to talk about the actual values that the pivot table will contain (since we've only talked about the headers of rows/columns until now):

The values that the pivot_table will contain are defined through the other two parameters, `values` and `aggfunc`: We select one or more columns of the initial DataFrame through the `values` parameter and these are aggregated in the corresponding cell of the resulting dataframe using the aggfunc fuction -- for each index and column, we will pick the values correspond to that cell and pass them to a function that will return the result. that As can be understood, the values must be different than index and columns (so all three sets of values, index and columns must not intersect). By default, the `values` and `aggfunc` parameters may be ommited - this will result in using average as the function and selecting all numerical columns in the values.

I know that this is difficult to understand so I'll give a simple example right away:


In [401]:
books_df.pivot_table(index=['decade', ], columns=['genre'], )

pages                                 year                       \
genre        Crime Fantasy Horror Thriller        Crime      Fantasy  Horror   
decade                                                                         
70's           NaN     NaN  631.0      NaN          NaN          NaN  1976.5   
80's    515.000000  339.25  756.0    423.5  1989.000000  1984.000000  1984.5   
90's    387.416667  606.50  529.0      NaN  1994.083333  1994.666667  1998.0   

                 
genre  Thriller  
decade           
70's        NaN  
80's     1984.5  
90's        NaN

As we can see we just passed `decade` as an index and `genre` as a column. We ommited values and aggfunc so the default values were used. What happened? Pandas found the corresponding entries for each cell, got their average and put it in that cell. This happened for all numerical columns (pages and year). For example, since there are no Crime genre books in the 70's we got a `NaN` to both the pages and year values. However, there are two Horror books, with 823 and 439 pages so their average is 631. The year column will also use the average, something that doesn't actually makes sense. So let's display only the pages:

In [402]:
books_df.pivot_table(index=['decade', ], columns=['genre'], values=['pages'])

pages                        
genre        Crime Fantasy Horror Thriller
decade                                    
70's           NaN     NaN  631.0      NaN
80's    515.000000  339.25  756.0    423.5
90's    387.416667  606.50  529.0      NaN

In [411]:
# Please notice that you can skip index or columns (but not both)
print books_df.pivot_table(index=['decade', ],)
print books_df.pivot_table(columns=['decade', ], )

             pages         year
decade                         
70's    631.000000  1976.500000
80's    470.111111  1984.777778
90's    464.052632  1994.473684
decade    70's         80's         90's
pages    631.0   470.111111   464.052632
year    1976.5  1984.777778  1994.473684


Let's now say that we actually wanted to have a meaningful value for the year, for example the first year we have a book for that genre/decade:

In [8]:
books_df.pivot_table(index=['decade', ], columns=['genre'], aggfunc=min )

author                                                 \
genre              Crime            Fantasy        Horror      Thriller   
decade                                                                    
70's                None               None  Stephen King          None   
80's        John Grisham   Michael Crichton  Stephen King  Stephen King   
90's    George Pelecanos  George R.R Martin  Stephen King          None   

                    name                                                     \
genre              Crime           Fantasy        Horror           Thriller   
decade                                                                        
70's                None              None   Salem's Lot               None   
80's      A Time to Kill             Congo            It  Different Seasons   
90's    A Firing Offense  A Clash of Kings  Bag of bones               None   

       pages                          year                            size  \
genre  Crime Fantasy Horror Thriller Crime Fantasy Horror Thriller   Crime   
decade                                                                       
70's    None    None    439     None  None    None   1975     None    None   
80's     515     224    374      320  1989    1980   1983     1982  medium   
90's     216     430    529     None  1991    1990   1998     None  medium   

                                 
genre  Fantasy  Horror Thriller  
decade                           
70's      None     big     None  
80's    medium     big   medium  
90's       big  medium     None

This is more interesting. It seems that since we didn't use the default aggfunc value but instead we passed our own, pandas autofilled values with all remaining values: Remember that our pool of data was *author, genre, name, pages, year, decade, size*, the genre and decade were used as an index/column so the remaining headers were used as values: author, name, pages, year, size! For the pages and year we can understand what happens: For example, for the Horror novels of the 80's, the one with the minimal pages is Pet Sematery with 374 pages. The same has also the minimal year (1983).  However, the one with the minimal name is `It` (since `I` is before `P` it just compares strings). The author is the same for both(`Stephen King`) and the minimum size is medium (since `small (s) > medium (m)`). Of course we could use values to actually define which values we wanted to see.

Another really interesting thing is to take a peek at which are the values that are passed to the aggregation function. For this, we can just use `tuple`:

In [9]:
# Please notice that for reasons unknown to me, if I used aggfunc=tuple it would throw an exception
books_df_tuples = books_df.pivot_table(index=['decade', ], columns=['genre'], aggfunc=lambda x: tuple(x))
books_df_tuples

author  \
genre                                               Crime   
decade                                                      
70's                                                 None   
80's                                      (John Grisham,)   
90's    (Michael Crichton, Michael Crichton, Michael C...   

                                                           \
genre                                             Fantasy   
decade                                                      
70's                                                 None   
80's    (Stephen King, Stephen King, Michael Crichton,...   
90's    (Stephen King, Stephen King, Michael Crichton,...   

                                                                    \
genre                         Horror                      Thriller   
decade                                                               
70's    (Stephen King, Stephen King)                          None   
80's    (Stephen King, Stephen King)  (Stephen King, Stephen King)   
90's                 (Stephen King,)                          None   

                                                     name  \
genre                                               Crime   
decade                                                      
70's                                                 None   
80's                                    (A Time to Kill,)   
90's    (Airframe, Rising Sun, Disclosure , The Firm, ...   

                                                           \
genre                                             Fantasy   
decade                                                      
70's                                                 None   
80's    (The Dark Tower: The Gunslinger, The Dark Towe...   
90's    (The Dark Tower III: The Waste Lands, The Dark...   

                                                               \
genre                     Horror                     Thriller   
decade                                                          
70's    (The Stand, Salem's Lot)                         None   
80's          (It, Pet Sematary)  (Misery, Different Seasons)   
90's             (Bag of bones,)                         None   

                                                    pages  \
genre                                               Crime   
decade                                                      
70's                                                 None   
80's                                               (515,)   
90's    (352, 385, 597, 432, 387, 496, 434, 414, 347, ...   

                                                                 \
genre                          Fantasy       Horror    Thriller   
decade                                                            
70's                              None   (823, 439)        None   
80's              (224, 400, 348, 385)  (1138, 374)  (320, 527)   
90's    (512, 787, 448, 430, 768, 694)       (529,)        None   

                                                     year  \
genre                                               Crime   
decade                                                      
70's                                                 None   
80's                                              (1989,)   
90's    (1996, 1992, 1994, 1991, 1992, 1994, 1995, 199...   

                                                                          \
genre                                Fantasy        Horror      Thriller   
decade                                                                     
70's                                    None  (1978, 1975)          None   
80's                (1982, 1987, 1980, 1987)  (1986, 1983)  (1987, 1982)   
90's    (1991, 1998, 1990, 1995, 1998, 1996)       (1998,)          None   

                                                     size  \
genre                                               Crime   
decade                                                 

In [10]:
# Dont worry about the ellipsis, the values are all there in each cell, for example
books_df_tuples['author']['Crime']['90\'s']

('Michael Crichton',
 'Michael Crichton',
 'Michael Crichton',
 'John Grisham',
 'John Grisham',
 'John Grisham',
 'John Grisham',
 'John Grisham',
 'John Grisham',
 'George Pelecanos',
 'George Pelecanos',
 'George Pelecanos')

In [11]:
# What if we wanted to only display the different authors for each genre and decade? Easy:
books_df.pivot_table(
    index=['decade', ], 
    columns=['genre'], 
    values='author',
    aggfunc=lambda x: tuple(set(x))
)

genre,Crime,Fantasy,Horror,Thriller
decade,,,,
70's,None,None,"(Stephen King,)",None
80's,"(John Grisham,)","(Stephen King, Michael Crichton)","(Stephen King,)","(Stephen King,)"
90's,"(John Grisham, Michael Crichton, George Peleca...","(Stephen King, George R.R Martin, Michael Cric...","(Stephen King,)",None


Notice that the inpout parameter that is passed to our aggunc is actually a `Series` so don't be alarmed if list operations are not working:

In [12]:
books_df.pivot_table(
    index=['decade', ], 
    columns=['genre'], 
    values='author',
    aggfunc=lambda x: type(x)
)

genre,Crime,Fantasy,Horror,Thriller
decade,,,,
70's,None,None,<class 'pandas.core.series.Series'>,None
80's,<class 'pandas.core.series.Series'>,<class 'pandas.core.series.Series'>,<class 'pandas.core.series.Series'>,<class 'pandas.core.series.Series'>
90's,<class 'pandas.core.series.Series'>,<class 'pandas.core.series.Series'>,<class 'pandas.core.series.Series'>,None


Before continuing, I'd like to present another two parameters that could be passed to the pivot_table: `fill_value` to define a value to display when no values are found to be aggregated for a cell and `margins` to enable or disable margin rows/columns to the left/bottom that will aggregate all values of that column: For example

In [13]:
books_df.pivot_table(
    index=['decade', ],
    columns=['genre'], values
    ='author', 
    aggfunc=lambda x: tuple(set(x)), 
    margins=True, 
    fill_value='-'
)

genre,Crime,Fantasy,Horror,Thriller,All
decade,,,,,
70's,-,-,"(Stephen King,)",-,"(Stephen King,)"
80's,"(John Grisham,)","(Stephen King, Michael Crichton)","(Stephen King,)","(Stephen King,)","(Stephen King, John Grisham, Michael Crichton)"
90's,"(John Grisham, Michael Crichton, George Peleca...","(Stephen King, George R.R Martin, Michael Cric...","(Stephen King,)",-,"(Stephen King, George R.R Martin, John Grisham..."
All,"(John Grisham, Michael Crichton, George Peleca...","(Stephen King, George R.R Martin, Michael Cric...","(Stephen King,)","(Stephen King,)","(Stephen King, George R.R Martin, John Grisham..."


Using our previous knowledge of multi column indexes, let's display the average number of pages each author writes for each decade and genre:

In [14]:
books_df.pivot_table(
    index=['decade', ],
    columns=['author', 'genre'], 
    values=['pages'], 
)

pages                                                  \
author George Pelecanos George R.R Martin John Grisham Michael Crichton   
genre             Crime           Fantasy        Crime            Crime   
decade                                                                    
70's                NaN               NaN          NaN              NaN   
80's                NaN               NaN   515.000000              NaN   
90's         268.333333             731.0   418.333333       444.666667   

                                             
author         Stephen King                  
genre  Fantasy      Fantasy Horror Thriller  
decade                                       
70's       NaN          NaN  631.0      NaN  
80's     366.5        312.0  756.0    423.5  
90's     439.0        649.5  529.0      NaN

In [15]:
# One interesting thing is that if we changed the order of the multi-columns we'd get the same data
books_df.pivot_table(
    index=['decade', ],
    columns=['genre', 'author'], 
    values=['pages'], 
)

pages                                                  \
genre             Crime                                         Fantasy   
author George Pelecanos John Grisham Michael Crichton George R.R Martin   
decade                                                                    
70's                NaN          NaN              NaN               NaN   
80's                NaN   515.000000              NaN               NaN   
90's         268.333333   418.333333       444.666667             731.0   

                                                                
genre                                      Horror     Thriller  
author Michael Crichton Stephen King Stephen King Stephen King  
decade                                                          
70's                NaN          NaN        631.0          NaN  
80's              366.5        312.0        756.0        423.5  
90's              439.0        649.5        529.0          NaN

In [16]:
# Or we can interchange index with columns to get the same data in a horizontal format
books_df.pivot_table(
    columns=['decade', ],
    index=['author', 'genre'], 
    values=['pages'], 
)

pages                   
decade                       70's   80's        90's
author            genre                             
George Pelecanos  Crime       NaN    NaN  268.333333
George R.R Martin Fantasy     NaN    NaN  731.000000
John Grisham      Crime       NaN  515.0  418.333333
Michael Crichton  Crime       NaN    NaN  444.666667
                  Fantasy     NaN  366.5  439.000000
Stephen King      Fantasy     NaN  312.0  649.500000
                  Horror    631.0  756.0  529.000000
                  Thriller    NaN  423.5         NaN

So, `Michael Crichton` was writing 445 pages for Crime novels and 439 pages for Fantasy novels on average at the 90's (of course this would be true if we had included all works of Michael Crichton).
In the previous table we can see that, for example for `George Pelecanos` only the `Crime` genre is displayed (since he's only Crime genre books in our database). Pandas automatically drops columns / lines where everything is empty -- if we for some reason wanted to display it, we'd use the `dropna=False` parameter:

In [17]:
books_df.pivot_table(
    index=['decade', ], 
    columns=['author', 'genre'], 
    values=['pages'], 
    dropna=False
)

pages                                                    \
author George Pelecanos                         George R.R Martin           
genre             Crime Fantasy Horror Thriller             Crime Fantasy   
decade                                                                      
70's                NaN     NaN    NaN      NaN               NaN     NaN   
80's                NaN     NaN    NaN      NaN               NaN     NaN   
90's         268.333333     NaN    NaN      NaN               NaN   731.0   

                                                                              \
author                 John Grisham                         Michael Crichton   
genre  Horror Thriller        Crime Fantasy Horror Thriller            Crime   
decade                                                                         
70's      NaN      NaN          NaN     NaN    NaN      NaN              NaN   
80's      NaN      NaN   515.000000     NaN    NaN      NaN              NaN   
90's      NaN      NaN   418.333333     NaN    NaN      NaN       444.666667   

                                                                     
author                         Stephen King                          
genre  Fantasy Horror Thriller        Crime Fantasy Horror Thriller  
decade                                                               
70's       NaN    NaN      NaN          NaN     NaN  631.0      NaN  
80's     366.5    NaN      NaN          NaN   312.0  756.0    423.5  
90's     439.0    NaN      NaN          NaN   649.5  529.0      NaN

In [396]:
# We can create any combination we want with our multi-index colums, for example let's see where each book belongs
# be decade / year / author and genre / size
books_df.pivot_table(
    index=['decade', 'year', 'author', 'name' ], 
    columns=['size', 'genre'], 
    values=['pages'], 
    aggfunc=lambda x: 'v',
    fill_value='',
)
# To remove the "pages" title, just index by it -- result['pages']

pages  \
size                                                                          big   
genre                                                                     Fantasy   
decade year author            name                                                  
70's   1975 Stephen King      Salem's Lot                                           
       1978 Stephen King      The Stand                                             
80's   1980 Michael Crichton  Congo                                                 
       1982 Stephen King      Different Seasons                                     
                              The Dark Tower: The Gunslinger                        
       1983 Stephen King      Pet Sematary                                          
       1986 Stephen King      It                                                    
       1987 Michael Crichton  Sphere                                                
            Stephen King      Misery                                                
                              The Dark Tower II: The Drawing of the Three           
       1989 John Grisham      A Time to Kill                                        
90's   1990 Michael Crichton  Jurassic Park                                         
       1991 John Grisham      The Firm                                              
            Stephen King      The Dark Tower III: The Waste Lands                   
       1992 George Pelecanos  A Firing Offense                                      
            John Grisham      The Pelican Brief                                     
            Michael Crichton  Rising Sun                                            
       1993 George Pelecanos  Nick's Trip                                           
       1994 John Grisham      The Chamber                                           
            Michael Crichton  Disclosure                                            
       1995 John Grisham      The Rainmaker                                         
            Michael Crichton  The Lost World                                        
       1996 George Pelecanos  The Big Blowdown                                      
            George R.R Martin A Game of Thrones                                 v   
            John Grisham      The Runaway Jury                                      
            Michael Crichton  Airframe                                              
       1998 George R.R Martin A Clash of Kings                                  v   
            John Grisham      The Street Lawyer                                     
            Stephen King      Bag of bones                                          
                              The Dark Tower IV: Wizard and Glass               v   

                                                                                  \
size                                                                               
genre                                                                     Horror   
decade year author            name                                                 
70's   1975 Stephen King      Salem's Lot                                          
       1978 Stephen King      The Stand                                        v   
80's   1980 Michael Crichton  Congo                                                
       1982 Stephen King      Different Seasons                                    
                              The Dark Tower: The Gunslinger                       
       1983 Stephen King      Pet Sematary                                         
       1986 Stephen King      It                                               v   
       1987 Michael Crichton  Sphere                                               
            Stephen King      Misery                                               
                              The Dark Tower II: The Drawing of the Three          
       19

One more advanced thing I'd like to cover here is that we could define multiple aggregate functions for each one of our values by passing a dictionary of `value:function` to the `aggfunc` parameter. For example, if we wanted to display 
* the sum of the pages that have been written
* the range of years for which we have books
* the names of the authors
* the name of one book we have
for each genre each decade, we could do something like this

In [35]:
def get_range(years):
    return '{0} - {1}'.format(min(years), max(years))

def get_names(authors):
    return ', '.join(set(authors))

def get_book(books):
    # Don't forget the the passed parameter is a Series so we use iloc to index it
    return books.iloc[0]
    
    
books_df.pivot_table(
    index=['decade',  ], 
    columns=['genre', ], 
    values=['author', 'pages', 'year', 'name'], 
    aggfunc={
        'author': get_names,
        'pages': sum,
        'year': get_range,
        'name': get_book,
        
    },
    fill_value='-'
)

year                                        pages          \
genre         Crime      Fantasy       Horror     Thriller Crime Fantasy   
decade                                                                     
70's              -            -  1975 - 1978            -     -       -   
80's    1989 - 1989  1980 - 1987  1983 - 1986  1982 - 1987   515    1357   
90's    1991 - 1998  1990 - 1998  1998 - 1998            -  4649    3639   

                                  name                                       \
genre  Horror Thriller           Crime                              Fantasy   
decade                                                                        
70's     1262        -               -                                    -   
80's     1512      847  A Time to Kill       The Dark Tower: The Gunslinger   
90's      529        -        Airframe  The Dark Tower III: The Waste Lands   

                               \
genre         Horror Thriller   
decade                          
70's       The Stand        -   
80's              It   Misery   
90's    Bag of bones        -   

                                                  author  \
genre                                              Crime   
decade                                                     
70's                                                   -   
80's                                        John Grisham   
90's    John Grisham, Michael Crichton, George Pelecanos   

                                                                         \
genre                                             Fantasy        Horror   
decade                                                                    
70's                                                    -  Stephen King   
80's                       Stephen King, Michael Crichton  Stephen King   
90's    Stephen King, George R.R Martin, Michael Crichton  Stephen King   

                      
genre       Thriller  
decade                
70's               -  
80's    Stephen King  
90's               -

Friends of pivot_table
======================

The `pivot_table` method has some friends -- these are functions that operate on DataFrame and can do reshaping but they are not as powerful as pivot_table. Let's introduce some of them:

In [200]:
# First, I'll create a DataFrame as an example:
df=books_df.pivot_table(index=['decade', ], columns=['genre', 'author'], values=['pages'], aggfunc=sum, )
# This df has a Multi-index in columns - first level is 'pages', second is the genres
# Notice that the levels parameter has three elements
print df.columns
# I'll make everything simpler by clearing out a level (just select the pages column)
df=df['pages']
# Notice that now, the levels parameter has two elements (['pages'] was removed)
print df.columns
df

MultiIndex(levels=[[u'pages'], [u'Crime', u'Fantasy', u'Horror', u'Thriller'], [u'George Pelecanos', u'George R.R Martin', u'John Grisham', u'Michael Crichton', u'Stephen King']],
           labels=[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 1, 1, 2, 3], [0, 2, 3, 1, 3, 4, 4, 4]],
           names=[None, u'genre', u'author'])
MultiIndex(levels=[[u'Crime', u'Fantasy', u'Horror', u'Thriller'], [u'George Pelecanos', u'George R.R Martin', u'John Grisham', u'Michael Crichton', u'Stephen King']],
           labels=[[0, 0, 0, 1, 1, 1, 2, 3], [0, 2, 3, 1, 3, 4, 4, 4]],
           names=[u'genre', u'author'])


genre             Crime                                         Fantasy  \
author George Pelecanos John Grisham Michael Crichton George R.R Martin   
decade                                                                    
70's                NaN          NaN              NaN               NaN   
80's                NaN        515.0              NaN               NaN   
90's              805.0       2510.0           1334.0            1462.0   

genre                                      Horror     Thriller  
author Michael Crichton Stephen King Stephen King Stephen King  
decade                                                          
70's                NaN          NaN       1262.0          NaN  
80's              733.0        624.0       1512.0        847.0  
90's              878.0       1299.0        529.0          NaN

Stack / unstack
---------------
These two operations move columns to indexes and vice-versa. Let's see what the [manual says](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.stack.html):

* stack: Pivot a level of the (possibly hierarchical) column labels, returning a DataFrame (or Series in the case of an object with a single level of column labels) having a hierarchical index with a new inner-most level of row labels. 
* unstack: Pivot a level of the (necessarily hierarchical) index labels, returning a DataFrame having a new level of column labels whose inner-most level consists of the pivoted index labels. If the index is not a MultiIndex, the output will be a Series (the analogue of stack when the columns are not a MultiIndex). The level involved will automatically get sorted.

I must confess that I was not able to comprehend the above! A more easy explanation is that:
* stack will re-arrange the values of the DataFrame so that the most inner column (the one at the bottom) will be converted to the most inner index (to the right)
* unstack will do the exactly opposite: Re-arrange the values of the DataFrame so that the most inner index (the one at the right) will be converted to the most inner column (to the bottom)

Also, stack and unstack do not really make sense. It would be much easier (at least to me) if stack was named `col_to_idx` and unstack was named `idx_to_col`.

In [204]:
# Before looking at examples of stack and unstack let's take a look at the index and columns of our dataframe
print "Index\n",df.index
print "Column\n",df.columns

Index
Index([u'70's', u'80's', u'90's'], dtype='object', name=u'decade')
Column
MultiIndex(levels=[[u'Crime', u'Fantasy', u'Horror', u'Thriller'], [u'George Pelecanos', u'George R.R Martin', u'John Grisham', u'Michael Crichton', u'Stephen King']],
           labels=[[0, 0, 0, 1, 1, 1, 2, 3], [0, 2, 3, 1, 3, 4, 4, 4]],
           names=[u'genre', u'author'])


In [206]:
stacked = df.stack()
print "Index\n",stacked.index
print "Column\n",stacked.columns
stacked

Index
MultiIndex(levels=[[u'70's', u'80's', u'90's'], [u'George Pelecanos', u'George R.R Martin', u'John Grisham', u'Michael Crichton', u'Stephen King']],
           labels=[[0, 1, 1, 1, 2, 2, 2, 2, 2], [4, 2, 3, 4, 0, 1, 2, 3, 4]],
           names=[u'decade', u'author'])
Column
Index([u'Crime', u'Fantasy', u'Horror', u'Thriller'], dtype='object', name=u'genre')


genre                      Crime  Fantasy  Horror  Thriller
decade author                                              
70's   Stephen King          NaN      NaN  1262.0       NaN
80's   John Grisham        515.0      NaN     NaN       NaN
       Michael Crichton      NaN    733.0     NaN       NaN
       Stephen King          NaN    624.0  1512.0     847.0
90's   George Pelecanos    805.0      NaN     NaN       NaN
       George R.R Martin     NaN   1462.0     NaN       NaN
       John Grisham       2510.0      NaN     NaN       NaN
       Michael Crichton   1334.0    878.0     NaN       NaN
       Stephen King          NaN   1299.0   529.0       NaN

We see that the `author` column (which was the most inner column) was moved to the right of the indexes. The rows (index) was converted to a multi-index  while the columns is a simple index now. 

In [213]:
# We can of course stack again -- this time we'll get a series since there are no more columns
stacked2 = stacked.stack()
print stacked2.index
stacked2

MultiIndex(levels=[[u'70's', u'80's', u'90's'], [u'George Pelecanos', u'George R.R Martin', u'John Grisham', u'Michael Crichton', u'Stephen King'], [u'Crime', u'Fantasy', u'Horror', u'Thriller']],
           labels=[[0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2], [4, 2, 3, 4, 4, 4, 0, 1, 2, 3, 3, 4, 4], [2, 0, 1, 1, 2, 3, 0, 1, 0, 0, 1, 1, 2]],
           names=[u'decade', u'author', u'genre'])


decade  author             genre   
70's    Stephen King       Horror      1262.0
80's    John Grisham       Crime        515.0
        Michael Crichton   Fantasy      733.0
        Stephen King       Fantasy      624.0
                           Horror      1512.0
                           Thriller     847.0
90's    George Pelecanos   Crime        805.0
        George R.R Martin  Fantasy     1462.0
        John Grisham       Crime       2510.0
        Michael Crichton   Crime       1334.0
                           Fantasy      878.0
        Stephen King       Fantasy     1299.0
                           Horror       529.0
dtype: float64

In [219]:
# unstack does the opposite operation
unstacked = df.unstack()
print unstacked.index
unstacked

MultiIndex(levels=[[u'Crime', u'Fantasy', u'Horror', u'Thriller'], [u'George Pelecanos', u'George R.R Martin', u'John Grisham', u'Michael Crichton', u'Stephen King'], [u'70's', u'80's', u'90's']],
           labels=[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3], [0, 0, 0, 2, 2, 2, 3, 3, 3, 1, 1, 1, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4], [0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2]],
           names=[u'genre', u'author', u'decade'])


genre     author             decade
Crime     George Pelecanos   70's         NaN
                             80's         NaN
                             90's       805.0
          John Grisham       70's         NaN
                             80's       515.0
                             90's      2510.0
          Michael Crichton   70's         NaN
                             80's         NaN
                             90's      1334.0
Fantasy   George R.R Martin  70's         NaN
                             80's         NaN
                             90's      1462.0
          Michael Crichton   70's         NaN
                             80's       733.0
                             90's       878.0
          Stephen King       70's         NaN
                             80's       624.0
                             90's      1299.0
Horror    Stephen King       70's      1262.0
                             80's      1512.0
                             90's       529.

We now see that the  that the `decade` column (which was the only index) was moved as the most inner to the columns -- however this also converts this DataFrame to a Series!

One interesting thing to notice is that a Series can only be unstack()ed since it has no columns (so stack can't work remember stack = col_to_idx)

In [228]:
unstacked.unstack()

decade                        70's    80's    90's
genre    author                                   
Crime    George Pelecanos      NaN     NaN   805.0
         John Grisham          NaN   515.0  2510.0
         Michael Crichton      NaN     NaN  1334.0
Fantasy  George R.R Martin     NaN     NaN  1462.0
         Michael Crichton      NaN   733.0   878.0
         Stephen King          NaN   624.0  1299.0
Horror   Stephen King       1262.0  1512.0   529.0
Thriller Stephen King          NaN   847.0     NaN

In [231]:
# One final comment is that stack and unstack can get a level parameter o inticate which
# index/column level we want to pivot
unstacked.unstack(level=0)

genre                      Crime  Fantasy  Horror  Thriller
author            decade                                   
George Pelecanos  70's       NaN      NaN     NaN       NaN
                  80's       NaN      NaN     NaN       NaN
                  90's     805.0      NaN     NaN       NaN
George R.R Martin 70's       NaN      NaN     NaN       NaN
                  80's       NaN      NaN     NaN       NaN
                  90's       NaN   1462.0     NaN       NaN
John Grisham      70's       NaN      NaN     NaN       NaN
                  80's     515.0      NaN     NaN       NaN
                  90's    2510.0      NaN     NaN       NaN
Michael Crichton  70's       NaN      NaN     NaN       NaN
                  80's       NaN    733.0     NaN       NaN
                  90's    1334.0    878.0     NaN       NaN
Stephen King      70's       NaN      NaN  1262.0       NaN
                  80's       NaN    624.0  1512.0     847.0
                  90's       NaN   1299.0   529.0       NaN

pivot
-----

The [`pivot`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.pivot.html) command will convert a column values to an index. This is similar like the `pivot_table` but does not aggregate the values and does not create multi-hierarchy indexes so you must be careful that each cell will contain only one value.

In [311]:
# We'll use the initial books_df DataFrame
books_df.pivot(index='name', columns='genre', values='year')
# Notice that we used 'name' as an index (to be sure that each cell will contain a single value)

genre,Crime,Fantasy,Horror,Thriller
name,,,,
A Clash of Kings,NaN,1998.0,NaN,NaN
A Firing Offense,1992.0,NaN,NaN,NaN
A Game of Thrones,NaN,1996.0,NaN,NaN
A Time to Kill,1989.0,NaN,NaN,NaN
Airframe,1996.0,NaN,NaN,NaN
Bag of bones,NaN,NaN,1998.0,NaN
Congo,NaN,1980.0,NaN,NaN
Different Seasons,NaN,NaN,NaN,1982.0
Disclosure,1994.0,NaN,NaN,NaN


In [333]:
# We could pivot by using name as a column
books_df.pivot(index='decade', columns='name', values='pages')


name,A Clash of Kings,A Firing Offense,A Game of Thrones,A Time to Kill,Airframe,Bag of bones,Congo,Different Seasons,Disclosure,It,...,The Dark Tower III: The Waste Lands,The Dark Tower IV: Wizard and Glass,The Dark Tower: The Gunslinger,The Firm,The Lost World,The Pelican Brief,The Rainmaker,The Runaway Jury,The Stand,The Street Lawyer
decade,,,,,,,,,,,,,,,,,,,,,
70's,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,823.0,NaN
80's,NaN,NaN,NaN,515.0,NaN,NaN,348.0,527.0,NaN,1138.0,...,NaN,NaN,224.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90's,768.0,216.0,694.0,NaN,352.0,529.0,NaN,NaN,597.0,NaN,...,512.0,787.0,NaN,432.0,430.0,387.0,434.0,414.0,NaN,347.0


What happens is that we got the values of one column and coverted these to a column/index and use another column's values as the values of the new DataFrame. So, in the first example the values of the genre column were converted to columns and inside each cell we put the page number. In the second example instead we converted the decade value to index and put the page number inside each cell. In both cases we used the name of the book to be sure that we would each cell will contain one value (remember that pivot cannot aggregate).

groupby
--------

The final command we'll talk about and is related to `pivot_table` is [`groupby`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html). This of course is related to the SQL group by method and should be easy to comprehend. The `groupby` gets a parameter that defines how to group the entries and returns a GroupBy object that contains the groups. The GroupBy object can be enumerated to get the groups and their data. It's interesting to take a look at the structure of each such object:

In [363]:
groupby_object = books_df.groupby(['decade', 'author'])
print type(groupby_object)
# Let's see what groupby-object contains
for x in groupby_object:
    print "type: ", type(x), "len: ", len(x) #len(x), type(x[0]), type(x[1]), x[0]
    print "first element of tuple", type(x[0]), x[0]
    print "second element of tuple", type(x[1])
    

<class 'pandas.core.groupby.DataFrameGroupBy'>
type:  <type 'tuple'> len:  2
first element of tuple <type 'tuple'> ("70's", 'Stephen King')
second element of tuple <class 'pandas.core.frame.DataFrame'>
type:  <type 'tuple'> len:  2
first element of tuple <type 'tuple'> ("80's", 'John Grisham')
second element of tuple <class 'pandas.core.frame.DataFrame'>
type:  <type 'tuple'> len:  2
first element of tuple <type 'tuple'> ("80's", 'Michael Crichton')
second element of tuple <class 'pandas.core.frame.DataFrame'>
type:  <type 'tuple'> len:  2
first element of tuple <type 'tuple'> ("80's", 'Stephen King')
second element of tuple <class 'pandas.core.frame.DataFrame'>
type:  <type 'tuple'> len:  2
first element of tuple <type 'tuple'> ("90's", 'George Pelecanos')
second element of tuple <class 'pandas.core.frame.DataFrame'>
type:  <type 'tuple'> len:  2
first element of tuple <type 'tuple'> ("90's", 'George R.R Martin')
second element of tuple <class 'pandas.core.frame.DataFrame'>
type:  <ty

So, from the above we can see that the GroupBy object contains a number of 2-element tuples. Each tuple contains (another tuple with) the columns that were used for groupping and the actual data of that group. Now, we could either use the enumeration I shown above to operate on each group or, better, to use some of the methods that the GroupBy object contains:

In [390]:
# get some statistics
print groupby_object.mean()
print groupby_object.sum()

# We can use the aggregate method to do anything we want
def year_aggr(x):
    return '{0}-{1}'.format(max(x), min(x))

def genre_aggr(x):
    return ', '.join(set(x))
# Notice each aggregate function will get a Series with the values

groupby_object.aggregate({'year':year_aggr, 'pages': sum, 'genre':genre_aggr})

                               pages         year
decade author                                    
70's   Stephen King       631.000000  1976.500000
80's   John Grisham       515.000000  1989.000000
       Michael Crichton   366.500000  1983.500000
       Stephen King       497.166667  1984.500000
90's   George Pelecanos   268.333333  1993.666667
       George R.R Martin  731.000000  1997.000000
       John Grisham       418.333333  1994.333333
       Michael Crichton   442.400000  1993.400000
       Stephen King       609.333333  1995.666667
                          pages   year
decade author                         
70's   Stephen King        1262   3953
80's   John Grisham         515   1989
       Michael Crichton     733   3967
       Stephen King        2983  11907
90's   George Pelecanos     805   5981
       George R.R Martin   1462   3994
       John Grisham        2510  11966
       Michael Crichton    2212   9967
       Stephen King        1828   5987


genre  pages       year
decade author                                                        
70's   Stephen King                          Horror   1262  1978-1975
80's   John Grisham                           Crime    515  1989-1989
       Michael Crichton                     Fantasy    733  1987-1980
       Stephen King       Fantasy, Horror, Thriller   2983  1987-1982
90's   George Pelecanos                       Crime    805  1996-1992
       George R.R Martin                    Fantasy   1462  1998-1996
       John Grisham                           Crime   2510  1998-1991
       Michael Crichton              Fantasy, Crime   2212  1996-1990
       Stephen King                 Fantasy, Horror   1828  1998-1991

In [393]:
# It's interesting to notice that the previous is exactly
# the same that we can do with this pivot_table command
books_df.pivot_table(index=['decade', 'author'], values=['genre', 'pages', 'year'], aggfunc={
        'genre': genre_aggr,
        'year': year_aggr,
        'pages': sum,
    })

genre  pages       year
decade author                                                        
70's   Stephen King                          Horror   1262  1978-1975
80's   John Grisham                           Crime    515  1989-1989
       Michael Crichton                     Fantasy    733  1987-1980
       Stephen King       Fantasy, Horror, Thriller   2983  1987-1982
90's   George Pelecanos                       Crime    805  1996-1992
       George R.R Martin                    Fantasy   1462  1998-1996
       John Grisham                           Crime   2510  1998-1991
       Michael Crichton              Fantasy, Crime   2212  1996-1990
       Stephen King                 Fantasy, Horror   1828  1998-1991

A real-world example
====================

To continue with a real-world example, I will use [MovieLens 100k](http://grouplens.org/datasets/movielens/) to represent some `pivot_table` operations. To load the data I've used the code already provided by the [three part series I already mentioned](http://www.gregreda.com/2013/10/26/working-with-pandas-dataframes/)

In [97]:
import os
import pandas as pd

path = 'C:/Users/serafeim/Downloads/ml-100k' # Change this to your own directory
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(os.path.join(path, 'u.user'), sep='|', names=u_cols, encoding='latin-1')
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(os.path.join(path, 'u.data'), sep='\t', names=r_cols, encoding='latin-1')
m_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
movies = pd.read_csv(os.path.join(path, 'u.item'), sep='|', names=m_cols, usecols=range(5), encoding='latin-1')
movie_ratings = pd.merge(movies, ratings)
lens = pd.merge(movie_ratings, users)

In [99]:
lens['movie_id']

0           1
1           4
2           5
3           7
4           8
5           9
6          11
7          12
8          15
9          17
10         19
11         21
12         22
13         23
14         24
15         25
16         28
17         30
18         31
19         32
20         42
21         44
22         45
23         47
24         48
25         49
26         50
27         54
28         55
29         56
         ... 
99970     332
99971     334
99972     338
99973     346
99974     682
99975     873
99976     877
99977     886
99978    1527
99979     272
99980     288
99981     294
99982     300
99983     310
99984     313
99985     322
99986     328
99987     333
99988     338
99989     346
99990     354
99991     362
99992     683
99993     689
99994     690
99995     748
99996     751
99997     879
99998     894
99999     901
Name: movie_id, dtype: int64

In [263]:
droplens = lens.drop(['video_release_date', 'imdb_url'], 1)
droplens['release_year']=droplens['release_date'].apply(lambda x: str(x).split('-')[2] if len(str(x).split('-'))>2 else 0)
droplens.head()

,movie_id,title,release_date,user_id,rating,unix_timestamp,age,sex,occupation,zip_code,release_year
0,1,Toy Story (1995),01-Jan-1995,308,4,887736532,60,M,retired,95076,1995
1,4,Get Shorty (1995),01-Jan-1995,308,5,887737890,60,M,retired,95076,1995
2,5,Copycat (1995),01-Jan-1995,308,4,887739608,60,M,retired,95076,1995
3,7,Twelve Monkeys (1995),01-Jan-1995,308,4,887738847,60,M,retired,95076,1995
4,8,Babe (1995),01-Jan-1995,308,5,887736696,60,M,retired,95076,1995


In [264]:
droplens[droplens['release_year'] > '1995'].pivot_table(
    columns=['rating', 'sex',],
    index=[ 'release_year'],
    values=[ 'age'], 
    fill_value=0,
    margins=True 
)

age                                              \
rating                1                     2                     3   
sex                   F          M          F          M          F   
release_year                                                          
1996          30.486239  30.373198  32.781437  31.576959  33.086124   
1997          30.333333  30.795053  31.926647  33.264409  32.638767   
1998          25.105263  31.028169  34.205882  36.059524  32.360000   
All           30.181609  30.593199  32.400000  32.404891  32.857241   

                                                                                
rating                           4                     5                   All  
sex                   M          F          M          F          M             
release_year                                                                    
1996          32.718188  32.757335  32.178464  30.494777  32.073340  32.126852  
1997          33.808467  33.224982  33.826004  32.225541  33.025606  33.086874  
1998          34.153061  32.845238  33.860656  33.785714  34.089431  33.403633  
All           33.208988  32.970278  32.986571  31.375121  32.611830  32.598420

In [265]:
droplens[droplens['release_year'] > '1995'].pivot_table(index=['rating'], values=['age'], columns=['sex'] ).dtypes

     sex
age  F      float64
     M      float64
dtype: object

How to use the pivot!

In [266]:
droplens[droplens['release_year'] > '1995'].head().pivot(index='movie_id', columns='age', values='rating')

age,60
movie_id,
15,3
17,4
21,3
22,4
23,5
